# Stage 13: Productization — Lecture Notebook

This notebook demonstrates how to prepare a financial engineering project for reuse, handoff, and basic deployment concepts. It includes examples of creating a model, pickling it, building an API, generating a webpage with a chart, launching a Streamlit dashboard, and demonstrating productization alternatives.

## 1. Generate and Train a Basic Model

In [ ]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
import joblib
import os


In [ ]:
# Create folders if they don't exist
os.makedirs('model', exist_ok=True)

# Generate synthetic data
X, y = make_regression(n_samples=100, n_features=2, noise=0.1, random_state=42)

# Train a basic model
model = LinearRegression()
model.fit(X, y)

# Save the model to disk
joblib.dump(model, 'model/model.pkl')

# Test that it works
print("Test prediction:", model.predict([[0.1, 0.2]]))

## 2. Reload Model and Verify Predictions

In [ ]:
# Load the model
loaded_model = joblib.load('model/model.pkl')
print("Reloaded model prediction:", loaded_model.predict([[0.1, 0.2]]))

## 3. Define Prediction Function

In [ ]:
def predict(features, round_prediction=False):
    """
    Predict using the loaded model.
    Optional parameter: round_prediction
    """
    pred = loaded_model.predict([features])[0]
    if round_prediction:
        pred = round(pred, 2)
    return pred

# Test the function
print("Function test:", predict([0.1, 0.2], round_prediction=True))

## 3b. Example of Modularization: Moving Functions to /src/

We will create a small utility function in `/src/utils.py` to demonstrate reusable code. 
This shows how project functions can be separated from the notebook for cleaner structure and better handoff.

In [ ]:
# Ensure /src/ exists
import os
os.makedirs('src', exist_ok=True)

# Write a simple utility function to /src/utils.py
utils_code = '''
def calculate_metrics(X, y):
    """
    Simple function to calculate regression metrics.
    Returns dict with mean and sum of outputs.
    """
    return {"mean_y": sum(y)/len(y), "sum_y": sum(y)}
'''

with open('src/utils.py', 'w') as f:
    f.write(utils_code)

print("Created /src/utils.py with calculate_metrics function.")

# Import and use the function
from src.utils import calculate_metrics

metrics = calculate_metrics([row[0] for row in X], y)
print("Metrics calculated from utility function:", metrics)

## 4. Save Flask App to External File (app.py) for External Launch

In [ ]:
flask_code = '''
from flask import Flask, request, jsonify, render_template_string
import joblib
import matplotlib.pyplot as plt
import io, base64

model = joblib.load('model/model.pkl')
app = Flask(__name__)

def predict(features):
    pred = model.predict([features])[0]
    return pred

@app.route('/predict', methods=['POST'])
def predict_default():
    data = request.get_json()
    features = data.get('features')
    return jsonify({'prediction': float(predict(features))})

@app.route('/predict/<float:input1>')
@app.route('/predict/<float:input1>/<float:input2>')
def predict_path(input1, input2=None):
    if input2 is None:
        features = [input1]
    else:
        features = [input1, input2]
    return jsonify({'prediction': float(predict(features))})

@app.route('/plot')
def plot_page():
    fig, ax = plt.subplots()
    ax.plot([0,1,2,3],[10,20,15,30])
    ax.set_title("Demo Plot")
    buf = io.BytesIO()
    fig.savefig(buf, format='png')
    buf.seek(0)
    img_base64 = base64.b64encode(buf.read()).decode('utf-8')
    buf.close()
    html = f"<html><body><h1>Model Output Plot</h1><img src='data:image/png;base64,{img_base64}'></body></html>"
    return render_template_string(html)

if __name__ == '__main__':
    app.run(debug=False)
'''

with open('app.py','w') as f:
    f.write(flask_code)

print("Flask app saved to app.py")

## 5. Launch Flask in External CMD Window (Windows Example)

In [ ]:
import os
# Windows command to launch Flask in separate terminal window
os.system("start cmd /k python app.py")
print("Flask server launched in external terminal window.")

## 6. Save and Launch Streamlit App Externally

In [ ]:
# !pip install streamlit

In [ ]:
streamlit_code = '''
import streamlit as st
import joblib

model = joblib.load('model/model.pkl')

st.title("Model Prediction Dashboard")

input_1 = st.number_input("Feature 1")
input_2 = st.number_input("Feature 2")

if st.button("Predict"):
    pred = model.predict([[input_1, input_2]])
    st.write(f"Prediction: {pred[0]}")
'''

with open('app_streamlit.py','w') as f:
    f.write(streamlit_code)

print("Streamlit app saved to app_streamlit.py")

# Launch Streamlit in external terminal (Windows example)
os.system("start cmd /k streamlit run app_streamlit.py")
print("Streamlit dashboard launched in external terminal window.")

## 7. Demonstrate Calling the API from Notebook

In [ ]:
import requests

# POST request with JSON payload
response = requests.post("http://127.0.0.1:5000/predict", json={"features":[0.1, 0.2]})
print("POST response:", response.json())

# GET request with path parameters
response2 = requests.get("http://127.0.0.1:5000/predict/0.1/0.2")
print("GET response:", response2.json())

## 8. Handoff-Ready Folder Structure & Productization Practices

In [ ]:
print('Example folder structure for productization:')
print('project_root/')
print('  /data/       # raw and processed data')
print('  /src/        # reusable functions and scripts')
print('  /notebooks/  # exploration and final notebooks')
print('  /reports/    # stakeholder summaries, charts, PDFs')
print('  /model/      # pickled model objects')
print('  README.md    # project overview, assumptions, instructions')

print('Ways to productize a project:')
print('- Expose model via an API (Flask, FastAPI)')
print('- Provide an interactive dashboard (Streamlit, Dash)')
print('- Generate automated batch scripts')
print('- Create clear documentation and README for handoff')
print('- Package code and outputs for reproducibility')
print('  - Code is modularized into stage driven libraries')
print('  - Functions are flexible and allow for many possibilities')
print('  - Analysis code runs from start to finish')
print('    - Code throws errors when stuck at a point that it needs human intervention')
print('    - Code logs and/or collects all relavant information as it runs')
print('  - Analyses can be run with a variety of data sets and inputs from start to finish')
print('    - Interaction and clear documentation is given to a user for this functionality')

## 9. Reading & Conceptual Notes

- Productization is about preparing code and artifacts for **reuse, handoff, and potential deployment**.
- Key practices include **modularity**, **reproducibility**, **documentation**, and **clear folder structures**.
- APIs allow other software systems to interact with your model; dashboards allow stakeholders to interact directly.
- Batch scripts or automated pipelines demonstrate repeatability.
- Handoff requires thinking like your future self or a teammate: 
  - what do they need to rerun, understand, or extend your work?
- Flask vs Streamlit: Flask exposes endpoints; Streamlit provides a web UI for demonstration.
- Even if not deploying, practicing these steps prepares you for real production environments.
- Mock plots and sample dashboards show how outputs can be packaged for stakeholders.
- Always validate your README and structure by trying to run the project from start to finish using fresh clone or environment.